In [1]:
from randomAgent import RandomAgentCPP 
from minimaxAgent import MinimaxAgentCPP 
import Agent
import random
from game_rule import DLLGameRule 
GR=DLLGameRule()
        
import ctypes

In [22]:
def competition(agent1,agent2,roundNumber):
    win={agent1.name:0,agent2.name:0}
    for R in range(roundNumber):
        numR=random.randint(0,1);
        agent1.reset()
        agent2.reset()
        if numR==0:
            agents=[agent1,agent2]
        else:
            agents=[agent2,agent1]
        
        state=GR.init_state()
        agent1.reset()
        agent2.reset()
        t=0
        while True:
            t+=1

            M=agents[t%2].bestmove()
            agents[0].playmove(M)
            agents[1].playmove(M)
            GR.next_state(state,GR.stringToAction(state,M))
            #print(M)
            if(GR.checkStatus(state)!=1):
                win[agents[(t%2)].name]+=1
                break
            #print("Eval:",agents[1-numR].numEval)
        print(win)
        GR.delBoard(state)
    

In [31]:
aR=MinimaxAgentCPP(depth=1,weight=[0,0,0,1,-25,30])
aS=RandomAgentCPP()
#pressureOnOpponentQueen
#pressureOnOwnQueen
#movableSelf
#movableOpp
#nearMyQueen
#nearOpposite



In [32]:
competition(aR,aS,100)

{'MINIMAX': 1, 'RANDOM': 0}
{'MINIMAX': 2, 'RANDOM': 0}
{'MINIMAX': 3, 'RANDOM': 0}
{'MINIMAX': 4, 'RANDOM': 0}
{'MINIMAX': 5, 'RANDOM': 0}
{'MINIMAX': 6, 'RANDOM': 0}
{'MINIMAX': 7, 'RANDOM': 0}
{'MINIMAX': 8, 'RANDOM': 0}
{'MINIMAX': 9, 'RANDOM': 0}
{'MINIMAX': 10, 'RANDOM': 0}
{'MINIMAX': 11, 'RANDOM': 0}
{'MINIMAX': 12, 'RANDOM': 0}
{'MINIMAX': 13, 'RANDOM': 0}
{'MINIMAX': 14, 'RANDOM': 0}
{'MINIMAX': 15, 'RANDOM': 0}
{'MINIMAX': 16, 'RANDOM': 0}
{'MINIMAX': 17, 'RANDOM': 0}
{'MINIMAX': 18, 'RANDOM': 0}
{'MINIMAX': 19, 'RANDOM': 0}
{'MINIMAX': 20, 'RANDOM': 0}
{'MINIMAX': 21, 'RANDOM': 0}
{'MINIMAX': 22, 'RANDOM': 0}
{'MINIMAX': 23, 'RANDOM': 0}
{'MINIMAX': 24, 'RANDOM': 0}
{'MINIMAX': 25, 'RANDOM': 0}
{'MINIMAX': 26, 'RANDOM': 0}
{'MINIMAX': 27, 'RANDOM': 0}
{'MINIMAX': 28, 'RANDOM': 0}
{'MINIMAX': 29, 'RANDOM': 0}
{'MINIMAX': 30, 'RANDOM': 0}
{'MINIMAX': 31, 'RANDOM': 0}
{'MINIMAX': 32, 'RANDOM': 0}
{'MINIMAX': 33, 'RANDOM': 0}
{'MINIMAX': 34, 'RANDOM': 0}
{'MINIMAX': 35, 'RANDOM

In [23]:
(ctypes.c_double * 6)


randomAgent.c_double_Array_6

In [24]:
weight=[1,2,3,4,5,6]

In [25]:
w=(ctypes.c_double * 6)()
for i in range(6):
    w[i]=weight[i]

In [27]:
w[1]


2.0